In [1]:
from PyQt5 import QtCore, QtGui, QtWidgets
from PyQt5.QtCore import QObject, pyqtSlot
from test import Ui_MainWindow
import sys
from djitellopy.tello import Tello
import socket
import threading
import pywifi
import time
wifi = pywifi.PyWiFi()

from psutil import process_iter
from signal import SIGTERM # or SIGKILL

for proc in process_iter():
    for conns in proc.connections(kind='inet'):
        if conns.laddr.port == 9010:
            proc.send_signal(SIGTERM)

for proc in process_iter():
    for conns in proc.connections(kind='inet'):
        if conns.laddr.port == 9011:
            proc.send_signal(SIGTERM)            


# IP and port of Tello
tello1_address = ('192.168.0.10', 8889)
tello2_address = ('192.168.0.11', 8889)

# IP and port of local computer
local1_address = ('', 9010)
local2_address = ('', 9011)

sock1 = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
sock2 = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)

# Bind to the local address and port
sock1.bind(local1_address)
sock2.bind(local2_address)

def send(message ,delay):
  # Try to send the message otherwise print the exception
  try:
    sock1.sendto(message.encode(), tello1_address)
    sock2.sendto(message.encode(), tello2_address)
    print("Sending message: " + message)
    print("Sending message: " + message)
  except Exception as e:
    print("Error sending: " + str(e))

  # Delay for a user-defined period of time
  time.sleep(delay)

# Receive the message from Tello
def receive():
  # Continuously loop and listen for incoming messages
  while True:
    # Try to receive the message otherwise print the exception
    try:
      response1, ip_address = sock1.recvfrom(128)
      response2, ip_address = sock2.recvfrom(128)
      print("Received message: from Tello EDU #1: " + response1.decode(encoding='utf-8'))
      print("Received message: from Tello EDU #2: " + response2.decode(encoding='utf-8'))
    except Exception as e:
      # If there's an error close the socket and break out of the loop
      sock1.close()
      sock2.close()
      print("Error receiving: " + str(e))
      break

# Create and start a listening thread that runs in the background
# This utilizes our receive functions and will continuously monitor for incoming messages
receiveThread = threading.Thread(target=receive)
receiveThread.daemon = True
receiveThread.start()

    

class MainWindowUIClass( Ui_MainWindow ):
    def __init__( self ):
        '''Initialize the super class
        '''
        super().__init__()

global tello1
global tello2

def drone1():
    send("command",3)

def drone2():
    send("command",3)
    
def takeoff():
    send("takeoff",3)
    print("taking off")
        
def land():
    send("land",3)
    print("landing")
def right():
    send("right",3)
    
def left():
    send("left 40",3)
    
def forward():
    send("forward 40",3)
def back():
    send("back 40",3)

def down():
    send("down 40",3)
def up():
    send("up 40",3)


def sendboth(message1,message2):
     # Try to send the message otherwise print the exception
    try:
        sock1.sendto(message1.encode(), tello1_address)
        sock2.sendto(message2.encode(), tello2_address)
        print("Sending message: " + message1)
        print("Sending message: " + message2)
    except Exception as e:
        print("Error sending: " + str(e))

def square():
    for i in range(6):
        sendboth("forward 100","forward 100")
  # Delay for a user-defined period of time
        time.sleep(4)
        sendboth("cw 90","cw 90")
        time.sleep(4)
        
def rectangle():
    for i in range(6):
        sendboth("forward 50","forward 100")
        time.sleep(4)
        sendboth("cw 90","cw 90")
        time.sleep(4)
        sendboth("forward 100","forward 50")
        time.sleep(4)
        sendboth("cw 90","cw 90")
        time.sleep(4)
def main():
    """
    This is the MAIN ENTRY POINT of our application.  The code at the end
    of the mainwindow.py script will not be executed, since this script is now
    our main program.   We have simply copied the code from mainwindow.py here
    since it was automatically generated by '''pyuic5'''.

    """
    app = QtWidgets.QApplication(sys.argv)
    MainWindow = QtWidgets.QMainWindow()
    ui = MainWindowUIClass()
    ui.setupUi(MainWindow)
    ui.takeoffbtn.clicked.connect(takeoff)
    ui.landbtn.clicked.connect(land)
    ui.upbtn.clicked.connect(up)
    ui.backbtn.clicked.connect(back)
    ui.downbtn.clicked.connect(down)
    ui.forwardbtn.clicked.connect(forward)
    ui.leftbtn.clicked.connect(left)
    ui.rightbtn.clicked.connect(right)
    ui.drone1.clicked.connect(drone1)
    ui.drone2.clicked.connect(drone2)
    ui.squarebtn.clicked.connect(square)
    ui.rectanglebtn.clicked.connect(rectangle)
    
    MainWindow.show()
    sys.exit(app.exec_())
main()

SystemExit: 0

C:\Users\umair\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3334: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
